- TRAIN TOGETHER

        def TRAIN_TOGETHER(original_data, classifier, demographic):
            ohe = OHE(original_data)
            df_model = TRAIN_MODEL(ohe, classifier)
            SUBGROUP_ERR(df_model, original_data, demographic)

    
- TRAIN SEPARATELY

        def TRAIN_SEPARATELY(original_data, demographic, classifier):
            data_extract = EXTRACT_DATA(original_data, demographic, i)
            ohe = OHE(data_extract)
            TRAIN_MODEL(ohe, classifier)


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, cross_val_predict, cross_validate
import sklearn.linear_model as linear_model

from sklearn.cross_validation import cross_val_score
import seaborn as sns 
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import confusion_matrix, classification_report, recall_score, precision_score, f1_score
from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

%matplotlib inline

/usr/local/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data_osteo = pd.read_csv('osteo_calcium_discretized.csv')
print(data_osteo.shape)  

(61022, 202)


/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (50) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# extract needed columns
osteo_extract_filter = data_osteo.filter(['Strata','sex','Race_Cat_combine', 'Age_combine','osteo_predict', 'BMI_Avg_imputed', 
     'Alcohol_Prior', 'Tobacco_Prior','Drug_antipsych_prior', 'Drug_Estrogens_prior', 'Drug_Glucocorticoids_prior',
    'Drug_Nsaids_prior', 'Drug_Opiates_prior', 'Drug_Thiazide_prior','Drug_Loop_Diuretic_Prior',
      'Drug_Pp_inhibitors_prior',  'Drug_Progesterone_prior',   'Drug_Seizure_prior',  'Drug_Ssris_prior',
 'Drug_Tc_antidepress_prior',  'HeartDisease_Prior',  'Liver_Prior', 'PulmDisease_Prior',  'CNS_Disease_Prior',                                               
 'Malignancy_Prior','Hyponatremia_Prior','Chronic_Hyponatremia', 'Recent_Hyponatremia', 
'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1', 'cal_avg_prior_cat', 'cal_closest_osteo_cat']) 

osteo_extract_filter.shape

(61022, 32)

## Functions of TRAIN_TOGETHER()

In [4]:
def OHE(data):
    data_1 = data.copy()
    
    numeric_subset = data_1.select_dtypes('number')
    categorical_subset = data_1.select_dtypes('object')

    # One hot encode
    categorical_subset = pd.get_dummies(categorical_subset) 

    # Join the two dataframes using concat. Make sure to use axis = 1 to perform a column bind
    data_2 = pd.concat([numeric_subset, categorical_subset], axis = 1)

    print('OHE shape: ', data_2.shape, '\n')
    print('Cols:', data_2.columns)
    return pd.DataFrame(data=data_2)

In [9]:
def TRAIN_MODEL(data, classifier):
    fold = KFold(10, shuffle = True, random_state = 12345)
    strata = data['Strata'].unique() 

    all_preds = np.full(data.shape[0], 100)
    probability = np.ones(data.shape[0])

    for i, (train_index, test_index) in enumerate(fold.split(strata)):

        train_index_strata = strata[train_index]
        test_index_strata = strata[test_index] 

        X_train = data.loc[data['Strata'].isin(train_index_strata)].drop('osteo_predict', axis = 1)
        X_test = data.loc[data['Strata'].isin(test_index_strata)].drop('osteo_predict', axis = 1)

        y_train = data.loc[data['Strata'].isin(train_index_strata)]['osteo_predict']
        y_test = data.loc[data['Strata'].isin(test_index_strata)]['osteo_predict']

        if classifier == 'RF':
            lr = RandomForestClassifier(n_estimators=100, random_state = 12345)
        
        if classifier == 'LR':
            lr = linear_model.LogisticRegression()     
            
        fit = lr.fit(X_train, y_train)

        y_pred = lr.predict(X_test)
        all_preds[y_test.index] = y_pred

        probs = lr.predict_proba(X_test)
        probability[y_test.index] = probs[:, 1]

    #     print(classification_report(y_test, y_pred))_3['Strata'].unique() 

    data['all_pred'] = all_preds
    data['probability'] = probability
    print('\nfinal result of the whole model: ')
    print(classification_report(data['osteo_predict'], data['all_pred']), '\n')
    print(confusion_matrix(data['osteo_predict'], data['all_pred']), '\n') 
    print('ROC_AUC of', classifier, ' ',roc_auc_score(data['osteo_predict'], data['probability']))
    dataframe = pd.DataFrame(data=data)
    return dataframe, lr

In [10]:
def SUBGROUP_ERR(OHE, original_data, demographic):
    
    race = ['Race_Cat_combine_black', 'Race_Cat_combine_others', 'Race_Cat_combine_white']
    age = ['Age_combine_from_30_to_50', 'Age_combine_from_50_to_70', 
           'Age_combine_from_70_to_80', 'Age_combine_less_than_30','Age_combine_more_than_80']
    sex = ['sex_F', 'sex_M']   

    subgroup_value = original_data[demographic].value_counts()
    print("subgroups:", subgroup_value)
    
    if demographic == 'Race_Cat_combine': 
        for i in race: 
            subgroup_1 = OHE.loc[OHE[i] == 1]
            print(i, classification_report(subgroup_1['osteo_predict'], subgroup_1['all_pred']))
            print(confusion_matrix(subgroup_1['osteo_predict'],subgroup_1['all_pred']))
            print('ROC_AUC of', i, roc_auc_score(subgroup_1['osteo_predict'], subgroup_1['probability']))
            print('----')
            
    elif demographic == 'Age_combine':
        for i in age: 
            subgroup_2 = OHE.loc[OHE[i] == 1]
            print(i, classification_report(subgroup_2['osteo_predict'], subgroup_2['all_pred']))
            print(confusion_matrix(subgroup_2['osteo_predict'],subgroup_2['all_pred']))
            print('ROC_AUC of',i, roc_auc_score(subgroup_2['osteo_predict'], subgroup_2['probability']))
            print('------')
            
    elif demographic == 'sex': 
        for i in sex:
            subgroup_3 = OHE.loc[OHE[i] == 1]
            print(i, classification_report(subgroup_3['osteo_predict'], subgroup_3['all_pred']))
            print(confusion_matrix(subgroup_3['osteo_predict'],subgroup_3['all_pred']))
            print('ROC_AUC of',i, roc_auc_score(subgroup_3['osteo_predict'], subgroup_3['probability']))
            print('-----')

In [11]:
def TRAIN_TOGETHER(original_data, classifier, demographic):
    ohe = OHE(original_data)
    print('')
    df_model, lr = TRAIN_MODEL(ohe, classifier)
    print('')
    SUBGROUP_ERR(df_model, original_data, demographic)

In [193]:
TRAIN_TOGETHER(osteo_extract_filter, 'LR', 'Race_Cat_combine')

OHE shape:  (61022, 39) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F', 'sex_M',
       'Race_Cat_combine_black', 'Race_Cat_combine_others',
       'Race_Cat_combine_white', 'Age_combine_from_30_to_50',
       'Age_combine_from_50_to_70', 'Age_combine_from_70_to_80',
       'Age_combine_less_than_

In [194]:
TRAIN_TOGETHER(osteo_extract_filter, 'LR', 'Age_combine')

OHE shape:  (61022, 39) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F', 'sex_M',
       'Race_Cat_combine_black', 'Race_Cat_combine_others',
       'Race_Cat_combine_white', 'Age_combine_from_30_to_50',
       'Age_combine_from_50_to_70', 'Age_combine_from_70_to_80',
       'Age_combine_less_than_

In [203]:
TRAIN_TOGETHER(osteo_extract_filter, 'RF', 'sex')

OHE shape:  (61022, 39) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F', 'sex_M',
       'Race_Cat_combine_black', 'Race_Cat_combine_others',
       'Race_Cat_combine_white', 'Age_combine_from_30_to_50',
       'Age_combine_from_50_to_70', 'Age_combine_from_70_to_80',
       'Age_combine_less_than_

In [ ]:
TRAIN_TOGETHER(osteo_extract_filter, 'RF', 'Race_Cat_combine')

In [ ]:
TRAIN_TOGETHER(osteo_extract_filter, 'RF', 'Age_combine')

## Train separate

In [13]:
def EXTRACT_DATA(data, col, value):
    data_extract = data.loc[data[col] == value]
    print(data[col].value_counts(), '\n')
    print ('extracting: ', value, 'of column: ', col)
#     print('after extracting: ', data_extract.shape) 

    data_extract_1 = data_extract.reset_index()
    data_extract_2 = data_extract_1.drop(['index', col], axis = 1)  
    print('new data shape', data_extract_2.shape)
    return pd.DataFrame(data=data_extract_2)

In [14]:
def TRAIN_SEPARATELY(original_data, demographic, classifier):
    title = original_data[demographic].value_counts()
    
    for i in title.index:
        data_extract = EXTRACT_DATA(original_data, demographic, i)
        print(' ')
        ohe = OHE(data_extract)
        print(' ')
        TRAIN_MODEL(ohe, classifier)
        print('\n ------------------')

In [197]:
TRAIN_SEPARATELY(osteo_extract_filter, 'sex', 'LR')

F    53852
M     7170
Name: sex, dtype: int64 

extracting:  F of column:  sex
new data shape (53852, 31)
 
OHE shape:  (53852, 37) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'Race_Cat_combine_black',
       'Race_Cat_combine_others', 'Race_Cat_combine_white',
       'Age_combine_from_30_to_50', 'Age_

In [198]:
TRAIN_SEPARATELY(osteo_extract_filter, 'Age_combine', 'LR')

from_50_to_70    25908
from_70_to_80    14712
more_than_80     12154
from_30_to_50     7264
less_than_30       984
Name: Age_combine, dtype: int64 

extracting:  from_50_to_70 of column:  Age_combine
new data shape (25908, 31)
 
OHE shape:  (25908, 34) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F

In [199]:
TRAIN_SEPARATELY(osteo_extract_filter, 'Race_Cat_combine', 'LR')

white     36438
black     17256
others     7328
Name: Race_Cat_combine, dtype: int64 

extracting:  white of column:  Race_Cat_combine
new data shape (36438, 31)
 
OHE shape:  (36438, 36) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F', 'sex_M',
       'Age_combine_from_30_to_50', 'Age_combine_from

In [ ]:
# RF

In [200]:
TRAIN_SEPARATELY(osteo_extract_filter, 'sex', 'RF')

F    53852
M     7170
Name: sex, dtype: int64 

extracting:  F of column:  sex
new data shape (53852, 31)
 
OHE shape:  (53852, 37) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'Race_Cat_combine_black',
       'Race_Cat_combine_others', 'Race_Cat_combine_white',
       'Age_combine_from_30_to_50', 'Age_

In [201]:
TRAIN_SEPARATELY(osteo_extract_filter, 'Age_combine', 'RF')

from_50_to_70    25908
from_70_to_80    14712
more_than_80     12154
from_30_to_50     7264
less_than_30       984
Name: Age_combine, dtype: int64 

extracting:  from_50_to_70 of column:  Age_combine
new data shape (25908, 31)
 
OHE shape:  (25908, 34) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F

In [202]:
TRAIN_SEPARATELY(osteo_extract_filter, 'Race_Cat_combine', 'RF')

white     36438
black     17256
others     7328
Name: Race_Cat_combine, dtype: int64 

extracting:  white of column:  Race_Cat_combine
new data shape (36438, 31)
 
OHE shape:  (36438, 36) 

Cols: Index(['Strata', 'osteo_predict', 'BMI_Avg_imputed', 'Alcohol_Prior',
       'Tobacco_Prior', 'Drug_antipsych_prior', 'Drug_Estrogens_prior',
       'Drug_Glucocorticoids_prior', 'Drug_Nsaids_prior', 'Drug_Opiates_prior',
       'Drug_Thiazide_prior', 'Drug_Loop_Diuretic_Prior',
       'Drug_Pp_inhibitors_prior', 'Drug_Progesterone_prior',
       'Drug_Seizure_prior', 'Drug_Ssris_prior', 'Drug_Tc_antidepress_prior',
       'HeartDisease_Prior', 'Liver_Prior', 'PulmDisease_Prior',
       'CNS_Disease_Prior', 'Malignancy_Prior', 'Hyponatremia_Prior',
       'Chronic_Hyponatremia', 'Recent_Hyponatremia',
       'Calcium_Avg_Prior_decile_1', 'Calcium_Closest_Osteo_decile_1',
       'cal_avg_prior_cat', 'cal_closest_osteo_cat', 'sex_F', 'sex_M',
       'Age_combine_from_30_to_50', 'Age_combine_from